In [141]:
# Import necessary modules

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from random import randint, shuffle
import random
import re
import json
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

In [51]:
def get_base_form(word):
    sanakirja_url = 'https://www.suomisanakirja.fi/' + word

    response=requests.get(sanakirja_url)

    soup = BeautifulSoup(response.text, 'html.parser')

    results = soup.find_all("p", {"class": "also"})

    if results[0].findAll(text=True)[0] == 'Tarkoititko:\n':
        tarkoititko = results[0].findAll(text=True)[1]
    else: tarkoititko = ''
    return tarkoititko

print(get_base_form('ihmisiä'))

#Liian suuri joukko ihmisiä meni konserttilavan lähelle. Syntyi paniikki. Paikalla oli noin 50 000 ihmistä.

In [56]:
word_baseform = 'pyhäinpäivä'
#word_baseform = 'tulitus'
#word_baseform = 'nopea'
#word_baseform = 'hän'

wikiurl = 'https://fi.wiktionary.org/wiki/' + word_baseform

response=requests.get(wikiurl)

soup = BeautifulSoup(response.text, 'html.parser')

results = str(soup.find_all("span", {"class": "toctext"})[1])

word_class = re.sub('<.*?>', '', results)

print(word_class)

Substantiivi


In [57]:
if word_class == 'Verbi':
    wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Verbitaivutus/suomi/'+ word_baseform
elif word_class == 'Substantiivi':
    wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform + '#Taivutus'
elif word_class == 'Adjektiivi':
    wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Adjektiivitaivutus/suomi/'+ word_baseform
elif word_class == 'Pronomini':
    wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform
else: wikiurl = 'https://fi.wiktionary.org'
            
table_class="wikitable sortable jquery-tablesorter"

response=requests.get(wikiurl)

soup = BeautifulSoup(response.text, 'html.parser')

wikitable=soup.find('table',{'class':"wikitable"})

if wikitable is None:
    table_contents = ''
else: table_contents = wikitable.findAll(text=True)

unwanted_chars = [word, '+ ', '–', '\n', '–\n', 'Taivutus\n', 'sijamuoto', 'nominatiivi', 'genetiivi', 'partitiivi', \
                          'akkusatiivi', 'sisäpaikallissijat', 'inessiivi', 'elatiivi', 'illatiivi', 'ulkopaikallissijat', \
                          'adessiivi', 'ablatiivi', 'allatiivi', 'essiivi', 'translatiivi', 'abessiivi', 'instruktiivi', \
                          'komitatiivi', 'positiivi', 'sijamuoto', 'yksikkö', 'monikko', 'kieliopilliset sijamuodot\n', \
                          'sisäpaikallissijat\n', 'ulkopaikallissijat\n', 'muut sijamuodot\n', 'omistusliite', 'monikko\n', \
                          'Positiivi', 'Komparatiivi', 'Superlatiivi', 'muut\n','(',')', ')\n', ';', 'Indikatiivi\n', 'preesens\n', \
                          'perfekti\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'minä\n', \
                          'sinä\n','hän\n','me\n','-"-\n','te', 'Te\n','he\n','passiivi','imperfekti\n','pluskvamperfekti\n', '; ']

negatives = ['en ', 'olen ', 'en ole ', 'et ', 'olet ', 'et ole ', 'ei ',  'on ', 'ei ole ', 'emme ', 'olemme ', 'emme ole ',\
                     'ette ', 'olette ',  'ette ole ', 'ette ole ', 'eivät ', 'ovat ', 'eivät ole ', 'olin ', 'en ollut ', 'olit ', \
                     'et ollut ', 'oli ',  'ei ollut ', 'olimme ', 'emme olleet ', 'olitte ', 'ette olleet ', 'ette olleet ',  'olivat ', \
                     'eivät olleet ']

if len(table_contents) > 0:
    cases = [char for char in table_contents if char not in unwanted_chars]
    
    cases = [char for char in cases if char not in negatives]

    cases = [''.join(e for e in case if e.isalnum()) for case in cases]

    cases = list(dict.fromkeys(cases))
            
print(cases)  

['kissa', 'kissat', 'kissan', 'kissojen', 'kissain', 'kissaa', 'kissoja', 'kissassa', 'kissoissa', 'kissasta', 'kissoista', 'kissaan', 'kissoihin', 'kissalla', 'kissoilla', 'kissalta', 'kissoilta', 'kissalle', 'kissoille', 'kissana', 'kissoina', 'kissaksi', 'kissoiksi', 'kissatta', 'kissoitta', 'kissoin', 'kissoine']


In [116]:
def practice():

    yle_url = 'https://yle.fi/uutiset/osasto/selkouutiset/'

    response=requests.get(yle_url)

    soup = BeautifulSoup(response.text, 'html.parser')

    results = soup.find_all("div", {"class": "text"})

    yle_articles = results[1].findAll(text=True)

    h3s = results[1].findAll('h3')
    headlines = []

    for h3 in h3s:
        temp = str(h3)
        temp = temp.replace('<h3>', '')
        temp = temp.replace('</h3>', '')
        headlines.append(temp)
        del temp

    unwanted_text = ['+ ', '–', '\n', '–\n', 'TV:n selkouutisilla on kesätauko. ', \
                      '(',')', ')\n', ';', '-"-\n', '; ']

    unwanted_text = unwanted_text + headlines

    article_list = [char for char in yle_articles if char not in unwanted_text]

    article_list = [sent for sent in article_list if not '\n' in sent]

    article_list = [sent_tokenize(sent) for sent in article_list]

    article_list = [sent for sublist in article_list for sent in sublist]

    article_list = [sent for sent in article_list if len(word_tokenize(sent))>1 ]

    pattern = RegexpTokenizer(r'\w+')

    # Split into sentences

    num = randint(0, len(article_list)-1)

    sentence = article_list[num]

    # Tokenize sentence, remove punctuation, numbers, and capitalized words (to avoid proper nouns).

    tokenized_sent = pattern.tokenize(sentence)

    tokenized_sent = [word for word in tokenized_sent if re.sub(r'[0-9]', '', word)]

    tokenized_sent = [word for word in tokenized_sent if not word[0].isupper()]

    num = randint(0, len(tokenized_sent)-1)

    word = tokenized_sent[num]

    fill_in_blank = sentence.replace(' '+word, ' _____ ')

    # if len(v.analyze(word)) > 0:
    if 1 > 0:

        # lemmatized = v.analyze(word)[0]
        #
        # word_baseform = lemmatized['BASEFORM']
        # word_class = lemmatized['CLASS']

        word_baseform = 'asia'
        word_class = 'nimisana'

        #Extract the base form
        #word_baseform = voikko_dict[0]['BASEFORM']
        cases = []

        wikiurl = ''

        if word_class == 'teonsana':
            wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Verbitaivutus/suomi/'+ word_baseform
        elif word_class == 'nimisana':
            wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform + '#Taivutus'
        elif word_class == 'laatusana':
            wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Adjektiivitaivutus/suomi/'+ word_baseform
        elif word_class == 'asemosana':
            wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform
        else: wikiurl = 'https://fi.wiktionary.org'

        # get the response in the form of html


        table_class="wikitable sortable jquery-tablesorter"

        response=requests.get(wikiurl)

        # parse data from the html into a beautifulsoup object

        soup = BeautifulSoup(response.text, 'html.parser')

        wikitable=soup.find('table',{'class':"wikitable"})

        if wikitable is None:
            table_contents = ''
        else: table_contents = wikitable.findAll(text=True)

        unwanted_chars = [word, '+ ', '–', '\n', '–\n', 'Taivutus\n', 'sijamuoto', 'nominatiivi', 'genetiivi', 'partitiivi', \
                          'akkusatiivi', 'sisäpaikallissijat', 'inessiivi', 'elatiivi', 'illatiivi', 'ulkopaikallissijat', \
                          'adessiivi', 'ablatiivi', 'allatiivi', 'essiivi', 'translatiivi', 'abessiivi', 'instruktiivi', \
                          'komitatiivi', 'positiivi', 'sijamuoto', 'yksikkö', 'monikko', 'kieliopilliset sijamuodot\n', \
                          'sisäpaikallissijat\n', 'ulkopaikallissijat\n', 'muut sijamuodot\n', 'omistusliite', 'monikko\n', \
                          'Positiivi', 'Komparatiivi', 'Superlatiivi', 'muut\n','(',')', ')\n', ';', 'Indikatiivi\n', 'preesens\n', \
                          'perfekti\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'minä\n', \
                          'sinä\n','hän\n','me\n','-"-\n','te', 'Te\n','he\n','passiivi','imperfekti\n','pluskvamperfekti\n', '; ']

        negatives = ['en ', 'olen ', 'en ole ', 'et ', 'olet ', 'et ole ', 'ei ',  'on ', 'ei ole ', 'emme ', 'olemme ', 'emme ole ',\
                     'ette ', 'olette ',  'ette ole ', 'ette ole ', 'eivät ', 'ovat ', 'eivät ole ', 'olin ', 'en ollut ', 'olit ', \
                     'et ollut ', 'oli ',  'ei ollut ', 'olimme ', 'emme olleet ', 'olitte ', 'ette olleet ', 'ette olleet ',  'olivat ', \
                     'eivät olleet ']

        if len(table_contents) > 0:
            cases = [char for char in table_contents if char not in unwanted_chars]

            cases = [char for char in cases if char not in negatives]

            cases = [''.join(e for e in case if e.isalnum()) for case in cases]

            cases = list(dict.fromkeys(cases))

        #set options from selected word and randomly selected cases
        if len(cases) > 0:
            num = randint(0, len(cases)-1)
            option_2 = cases[num].replace('-', '').replace('(', '').replace(')', '')

            num = randint(0, len(cases)-1)
            option_3 = cases[num].replace('-', '').replace('(', '').replace(')', '')

            options = [word, option_2, option_3]

            options = random.sample(options, len(options))

            return fill_in_blank, options, word
        else: return practice()
    else: return practice()

In [130]:
practice()

('Silloin polttoaine _____  tuleen ja räjähti.',
 ['syttyi', 'asian', 'asioihin'],
 'syttyi')

In [7]:
word = 'tulivat'
url = 'https://fi.wiktionary.org/wiki/Toiminnot:Haku?search='+word+'&go=Siirry&ns0=1'

print(url)

https://fi.wiktionary.org/wiki/Toiminnot:Haku?search=tulivat&go=Siirry&ns0=1


search on wiktionary
if result is Hakutulokset, url = url of first result,
go to url, get word type
else go to url get word type
if word type is Verbi, Adjektiivi, Konjunktio
then wikiurl as originally done

list of basic verbs

get taivutus from sanakirja

store locally

if word in list of basic words, pull forms from there

else search on sanakirja




In [175]:
sample = ["olla","ja","ei","se","että","joka","vuosi","hän","myös","saada","mutta","tämä","voida","tulla","Suomi","tehdä","kun","pitää","mukaan","uusi","jo","kuin","hyvä","ne","sanoa","kaikki","markka","nyt","suuri","kertoa","tai","vain","paljon","muu","vielä","niin","oma","sekä","mikä","maa","kaksi","jos","asia","yksi","viime","he","prosentti","ihminen","kuitenkin","jälkeen","mies","kanssa","aika","noin","hyvin","ensimmäinen","jäädä","työ","ottaa","me","antaa","mieli","Helsinki","haluta","alkaa","EU","itse","moni","käydä","kaupunki","kolme","miljoona","osa","STT","alue","eikä","päästä","vaikka","lapsi","päivä","hallitus","toimia","sama","koko","pieni","mennä","ensi","kunta","toinen","sillä","joutua","käyttää","Tampere","kuulua","mikään","nainen","toinen","viikko","esimerkiksi","lähteä","eli","tarvita","yli","koska","Venäjä","Turku","luku","vanha","nousta","nähdä","kerta","jokin","todeta","aikana","yritys","minä","nämä","seurata","tärkeä","alku","paikka","vaan","tilanne","presidentti","tieto","loppu","tietää","tapa","maailma","aina","määrä","maksaa","vaatia","puheenjohtaja","löytyä","lähes","tuoda","uskoa","esittää","ennen", "neljä","Oulu","usea","poliisi","kisa","enää","sitten","auto","odottaa","toiminta","eri","entinen","näyttää","syntyä","Eurooppa","tulos","raha","päättää","saattaa","päätös","suomalainen","käyttö","aloittaa","syy","sitten","kasvaa","elämä","järjestää","johtaa","kilpailu","koulu","kesä","vuotias","ongelma","yhtiö","valita","joukkue","viisi","nuori","vähän","mahdollisuus","muutama","kausi","ainakin","jatkaa","suomalainen","jopa","kymmenen","voittaa","valtio","pelata","viedä","kuva","sopia","viimeinen","nimi","puoli","mukaan","nykyinen","arvioida","edelleen","mukana","juuri","joukko","pitkä","sellainen","johtaja","katsoa","vastata","näin","lisäksi","riittää","kansainvälinen","ajaa","kova","tuntea","Ruotsi","Yhdysvallat","puhua","rakentaa","pystyä","tila","apu","lisäksi","vasta","miten","ryhmä","hetki","tehtävä","puolue","liittyä","Kosovo","tapahtua","tavoite","voitto","matka","kauppa","ottelu","sopimus","jatkua","maanantai","hinta","kyllä","lisätä","sunnuntai","kuten","taas","sarja","perjantai","puoli","lauantai","jättää","Saksa","yhteistyö","jäsen","laskea","tutkimus","kuitenkaan","yrittää","tuntua","Joensuu","syksy","vaikea","torstai","vaikuttaa","nuori","mahdollinen","tiistai","keskiviikko","kysymys","ilmoittaa","aivan","löytää","talo","kuukausi","pyrkiä","tarjota","kannattaa","kolmas","ohjelma","ratkaisu","vastaan","peli","hoitaa","kirja","hyväksyä","myöntää","tuki","Jyväskylä","poika","esitys","muun muassa","siirtyä","aiheuttaa","aikoa","hakea","muuttaa","pari","edustaja","ala","myydä","nostaa","yhä","tie","perhe","kestää","pysyä","metri","muistuttaa","ilman","korkea","jokainen","elää","osallistua","liian","koskea","onnistua","oikeus","Matti","iso","asema","elokuva","voima","tarkoitus","edes","maali","toivoa","johto","muutos","silloin","ehkä","selvittää","Pekka","erilainen","tunti","tapahtuma","joten","hanke","yhteinen","Reuters","kevät","piste","kokous","vesi","liitto","tulevaisuus","ääni","kuusi","palvelu","juttu","kunto","taso","pääministeri","muuttua","usein","tulo","keskusta","joku","sata","jolloin","Nato","näkyä","työntekijä","valmis","osuus","samalla","käsi","kautta","heti","miljardi","poliittinen","ehtiä","asua","osoittaa","lehti","siellä","aiemmin","ostaa","asiakas","tällainen","henkilö","sivu","kirjoittaa","raja","toimitusjohtaja","musiikki","teos","kasvu","talous","sota","erityisesti","koti","perustaa","jakaa","kokea","kilometri","palata","asukas","korostaa","käytäntö","paha","luoda","vuoksi","vaali","Kari","kirkko","suunnitella","hieman","minuutti","keskustelu","päättyä","tutkia","oikea","tapaus","siis","osata","eduskunta","ero","Lipponen","arvo","henki","vahva","yleensä","tukea","molemmat","neuvottelu","seura","vaihe","kuulla","tarkoittaa","pankki","tuhat","kuinka","vetää","kuka","Juha","koulutus","lisää","jotta","miljoona","selvä","tuottaa","nopeasti","ainoa","myöhemmin","silti","työpaikka","vaikka","rakentaminen","ymmärtää","täysin","sijaita","käsitellä","helppo","Mika","merkitä","suhde","historia","komissio","laki","selvästi","pelaaja","tänään","takia","ratkaista","kello","muuten","ihan","näyttely","yhteys","todella","kokemus","ehdokas","laaja","kulua","asunto","kehitys","osake","huomio","Ranska","äiti","yleisö","siksi","etsiä","asettaa","huono","myynti","kiinni","pohtia","hoito","tarve","yleinen","ryhtyä","erittäin","tuttu","luvata","ajatella","suunta","kukaan","lupa","ajatus","tosin","hankkia","kulkea","kysyä","tyttö","miettiä","varten","edellinen","avata","yhdessä","saapua","kohde","muistaa","SM","tuote","kone","välinen","opettaja","epäillä","ilta","Nokia","maakunta","markkinat","Timo","yhteensä","kieli","täällä","Ahtisaari","vuosikymmen","tekijä","tavallinen","kansa","seitsemän","merkittävä","yö","sana","huolimatta","asti","vaihtoehto","TV","puuttua","auttaa","merkitys","valmistua","isä","runsas","saakka","Jugoslavia","Jukka","koskaan","YK","kerätä","yhtä","kuolla","liike","takaisin","suunnitelma","esiintyä","vakuuttaa","tuotanto","valmentaja","malli","parlamentti","kansalainen","pää","toteuttaa","kuvata","vanhempi","esitellä","aihe","talvi","vauhti","yhteiskunta","Kiina","eräs","siirtää","lähettää","lukea","varmasti","kahdeksan","toisaalta","täyttää","meno","vahvistaa","vaikutus","perinteinen","Hannu","rakennus","taloudellinen","kehittää","pois","edustaa","pitkään","kylä","kanta","nopea","opiskelija","vai","tavata","kannalta","huomata","varsin","Jari","metsä","lisääntyä","liikenne","työskennellä","kovin","hallita","tutkija","pyytää","miksi","ensin","laji","ohjata","oikein","vastuu","kansanedustaja","rooli","alle","lähinnä","tyytyväinen","Norja","järjestö","paikallinen","kärsiä","puu","etu","liikkua","professori","luokka","täytyä","linja","julkaista","istua","elokuu","joskus","Paavo","tuo","Martti","menettää","viettää","panna","selvitä","sattua","Antti","soittaa","politiikka","seutu","Aho","kiinnostaa","keskittyä","oppilas","kierros","kesäkuu","piiri","avoin","valinta","varsinainen","euro","puhe","kulttuuri","perustua","kaikkiaan","kutsua","ura","huoli","jälleen","valtuusto","Pohjois-Karjala","kohti","vastaan","Heikki","arvella","Viro","julkinen","liiga","Kokoomus","kehittäminen","pelätä","tuore","hävitä","omistaja","silmä","projekti","yhdistys","nousu","uhata","viranomainen","kenttä","erä","mennessä","aikaisin","tarina","tiukka","teatteri","väki","omistaa","kerran","tekeminen","täysi","puolesta","Hyvinkää","tässä","yllättää","huomauttaa","keskeinen","yrittäjä","keskustella","kotimainen","ministeri","varsinkin","tietysti","kustannus","sairaala","johtua","AFP","syyskuu","kansallinen","merkki","välillä","tuomita","keskus","lopettaa","lokakuu","Internet","Markku","oppia","Seppo","yli","sekunti","lopulta","onni","mikäli","parantaa","luonto","valta","ympäristö","pohja","todellinen","myötä","kantaa","toukokuu","keino","taiteilija","aika","Erkki","uudelleen","valmistaa","tehdas","Mikko","edetä","perustella","kieltää","suoraan","toimittaja","lyhyt","maaliskuu","nykyisin","estää","pian","ruoka","Lappi","toistaiseksi","takana","heinäkuu","tammikuu","virka","edellyttää","pelkkä","yhdistää","kriisi","Moskova","mainita","neljäs","vara","varma","tausta","helposti","joulukuu","Aamulehti","kallis","yliopisto","selvitys","vaihtaa","lääkäri","vähentää","koota","dollari","siten","Turkki","sija","mestaruus","Italia","marraskuu", "saavuttaa","SDP","sijoittaa","tilaisuus","olo","ystävä","jalka","palkinto","konsertti","Esko","rauha","katsoja","Jorma","pakolainen","ulkomaa","Niinistö","palkka","Keski-Suomi","vapaa","juhla","levy","vajaa","järjestelmä","toimi","julkisuus","sulkea","määrätä","teko","lasku","Halonen","kyse","vieras","kaivata","ennen","potilas","alla","ovi","äänestää","pudota","budjetti","laitos","varata","lyödä","tarkka","sijoittua","näytelmä","Israel","menestys","arvio","rata","kirjailija","voittaja","kappale","turvallisuus","nimittää","ilmestyä","lähtö","Britannia","ainakaan","ranta","televisio","luopua","ulkoministeri","Janne","heittää","esimerkki","herättää","etenkin","teksti","vakava","Lahti","taistelu","sujua","Salo","vaatimus","sisältää","seurakunta","kaunis","tekninen","kurssi","armeija","työryhmä","vaikeus","pois","ehto","taide","Kuopio","ehdotus","korvata","Mäkinen","Jeltsin","jälki","katu","ilmeisesti","muoto","tappio","syyttää","jatko","Hämeenlinna","näkemys","yhdeksän","väittää","mestari","virallinen","tuskin","suhtautua","syödä","teollisuus","kilo","käsittely","peräti","ihmetellä","vastustaa","vaimo","nauttia","tunne","säilyä","USA","helmikuu","kohdata","ote","huhtikuu","verrattuna","uhri","idea","suosia","esillä","ohella","rahoitus","reilu","kertyä","viikonloppu","liikevaihto","kärki","laulu","eläke","laatu","Risto","muodostaa","koira","vähintään","runsaasti","ampua","vastaus","seuraus","mukana","loppua","havaita","ylittää","joulu","painaa","työttömyys","kotimaa","lähtien","yksityinen","väli","toki","lista","Karjalainen","kehittyä","Häkkinen","iskeä","valo","jakso","kuluttua","välillä","vuosituhat","raskas","Riitta","toteutua","Venäläinen","neuvotella","palkita","vastaava","kilpailla","lausunto","sinne","tietty","julkistaa","kaupunginhallitus","virhe","Ilkka","asiantuntija","kuljettaja","eurooppalainen"]

common_words = [word for word in sample if not word[0].isupper()]

print(common_words)
    

['olla', 'ja', 'ei', 'se', 'että', 'joka', 'vuosi', 'hän', 'myös', 'saada', 'mutta', 'tämä', 'voida', 'tulla', 'tehdä', 'kun', 'pitää', 'mukaan', 'uusi', 'jo', 'kuin', 'hyvä', 'ne', 'sanoa', 'kaikki', 'markka', 'nyt', 'suuri', 'kertoa', 'tai', 'vain', 'paljon', 'muu', 'vielä', 'niin', 'oma', 'sekä', 'mikä', 'maa', 'kaksi', 'jos', 'asia', 'yksi', 'viime', 'he', 'prosentti', 'ihminen', 'kuitenkin', 'jälkeen', 'mies', 'kanssa', 'aika', 'noin', 'hyvin', 'ensimmäinen', 'jäädä', 'työ', 'ottaa', 'me', 'antaa', 'mieli', 'haluta', 'alkaa', 'itse', 'moni', 'käydä', 'kaupunki', 'kolme', 'miljoona', 'osa', 'alue', 'eikä', 'päästä', 'vaikka', 'lapsi', 'päivä', 'hallitus', 'toimia', 'sama', 'koko', 'pieni', 'mennä', 'ensi', 'kunta', 'toinen', 'sillä', 'joutua', 'käyttää', 'kuulua', 'mikään', 'nainen', 'toinen', 'viikko', 'esimerkiksi', 'lähteä', 'eli', 'tarvita', 'yli', 'koska', 'luku', 'vanha', 'nousta', 'nähdä', 'kerta', 'jokin', 'todeta', 'aikana', 'yritys', 'minä', 'nämä', 'seurata', 'tärkeä', '

In [176]:
wordbank = {}

#common_words = ['ei', 'kaupunginhallitus', 'virhe', 'asiantuntija', 'kuljettaja', 'eurooppalainen']

for word in common_words:
    
    word_baseform = word

    wikiurl = 'https://fi.wiktionary.org/wiki/' + word_baseform

    response=requests.get(wikiurl)

    soup = BeautifulSoup(response.text, 'html.parser')

    results = str(soup.find_all("span", {"class": "toctext"})[1])
    
    print(word, ": ", results)

    word_class = re.sub('<.*?>', '', results)

    if word_class == 'Verbi':
        wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Verbitaivutus/suomi/'+ word_baseform
    elif word_class == 'Substantiivi':
        wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform + '#Taivutus'
    elif word_class == 'Adjektiivi':
        wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Adjektiivitaivutus/suomi/'+ word_baseform
    elif word_class == 'Pronomini':
        wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform
    else: wikiurl = 'https://fi.wiktionary.org'
            
    table_class="wikitable sortable jquery-tablesorter"

    response=requests.get(wikiurl)

    soup = BeautifulSoup(response.text, 'html.parser')

    wikitable=soup.find('table',{'class':"wikitable"})

    if wikitable is None:
        table_contents = ''
    else: table_contents = wikitable.findAll(text=True)

    unwanted_chars = [word, '+ ', '–', '\n', '–\n', 'Taivutus\n', 'sijamuoto', 'nominatiivi', 'genetiivi', 'partitiivi', \
                          'akkusatiivi', 'sisäpaikallissijat', 'inessiivi', 'elatiivi', 'illatiivi', 'ulkopaikallissijat', \
                          'adessiivi', 'ablatiivi', 'allatiivi', 'essiivi', 'translatiivi', 'abessiivi', 'instruktiivi', \
                          'komitatiivi', 'positiivi', 'sijamuoto', 'yksikkö', 'monikko', 'kieliopilliset sijamuodot\n', \
                          'Sisäpaikallissijat\n', 'sisäpaikallissijat\n', 'Ulkopaikallissijat\n', 'ulkopaikallissijat\n', 'muut sijamuodot\n', 'omistusliite', 'monikko\n', \
                          'Positiivi', 'Komparatiivi', 'Superlatiivi', 'muut\n','(',')', ')\n', ';', 'Indikatiivi\n', 'preesens\n', \
                          'perfekti\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'minä\n', \
                          'sinä\n','hän\n','me\n','-"-\n','te', 'Te\n','he\n','passiivi','imperfekti\n','pluskvamperfekti\n', '; ']

    negatives = ['en ', 'olen ', 'en ole ', 'et ', 'olet ', 'et ole ', 'ei ',  'on ', 'ei ole ', 'emme ', 'olemme ', 'emme ole ',\
                     'ette ', 'olette ',  'ette ole ', 'ette ole ', 'eivät ', 'ovat ', 'eivät ole ', 'olin ', 'en ollut ', 'olit ', \
                     'et ollut ', 'oli ',  'ei ollut ', 'olimme ', 'emme olleet ', 'olitte ', 'ette olleet ', 'ette olleet ',  'olivat ', \
                     'eivät olleet ']

    if len(table_contents) > 0:
        cases = [char for char in table_contents if char not in unwanted_chars]

        cases = [char for char in cases if char not in negatives]

        cases = [''.join(e for e in case if e.isalnum()) for case in cases]

        cases = list(dict.fromkeys(cases))
    
        wordbank[word] = word
    
    else: cases = ''
        
    for case in cases:
        
        wordbank[case] = word
            
with open("wordbank.json", "w") as outfile:
    json.dump(wordbank, outfile)    

olla :  <span class="toctext">Verbi</span>
ja :  <span class="toctext">Lyhenne</span>
ei :  <span class="toctext">Interjektio</span>
se :  <span class="toctext">Lyhenne</span>
että :  <span class="toctext">Konjunktio</span>
joka :  <span class="toctext">Pronomini</span>
vuosi :  <span class="toctext">Substantiivi</span>
hän :  <span class="toctext">Pronomini</span>
myös :  <span class="toctext">Adverbi</span>
saada :  <span class="toctext">Verbi</span>
mutta :  <span class="toctext">Konjunktio</span>
tämä :  <span class="toctext">Pronomini</span>
voida :  <span class="toctext">Verbi</span>
tulla :  <span class="toctext">Verbi</span>
tehdä :  <span class="toctext">Verbi</span>
kun :  <span class="toctext">Konjunktio</span>
pitää :  <span class="toctext">Verbi</span>
mukaan :  <span class="toctext">Adverbi</span>
uusi :  <span class="toctext">Adjektiivi</span>
jo :  <span class="toctext">Adverbi</span>
kuin :  <span class="toctext">Adverbi</span>
hyvä :  <span class="toctext">Adjektiivi<

mahdollisuus :  <span class="toctext">Substantiivi</span>
muutama :  <span class="toctext">Pronomini</span>
kausi :  <span class="toctext">Substantiivi</span>
ainakin :  <span class="toctext">Adverbi</span>
jatkaa :  <span class="toctext">Verbi</span>
suomalainen :  <span class="toctext">Adjektiivi</span>
jopa :  <span class="toctext">Adverbi</span>
kymmenen :  <span class="toctext">Numeraali</span>
voittaa :  <span class="toctext">Verbi</span>
valtio :  <span class="toctext">Substantiivi</span>
pelata :  <span class="toctext">Verbi</span>
viedä :  <span class="toctext">Verbi</span>
kuva :  <span class="toctext">Substantiivi</span>
sopia :  <span class="toctext">Substantiivi</span>
viimeinen :  <span class="toctext">Adjektiivi</span>
nimi :  <span class="toctext">Substantiivi</span>
puoli :  <span class="toctext">Adjektiivi</span>
mukaan :  <span class="toctext">Adverbi</span>
nykyinen :  <span class="toctext">Adjektiivi</span>
arvioida :  <span class="toctext">Verbi</span>
edelleen : 

tulo :  <span class="toctext">Substantiivi</span>
keskusta :  <span class="toctext">Substantiivi</span>
joku :  <span class="toctext">Pronomini</span>
sata :  <span class="toctext">Numeraali</span>
jolloin :  <span class="toctext">Adverbi</span>
näkyä :  <span class="toctext">Substantiivi</span>
työntekijä :  <span class="toctext">Substantiivi</span>
valmis :  <span class="toctext">Adjektiivi</span>
osuus :  <span class="toctext">Substantiivi</span>
samalla :  <span class="toctext">Adverbi</span>
käsi :  <span class="toctext">Substantiivi</span>
kautta :  <span class="toctext">Postpositio</span>
heti :  <span class="toctext">Adverbi</span>
miljardi :  <span class="toctext">Numeraali</span>
poliittinen :  <span class="toctext">Adjektiivi</span>
ehtiä :  <span class="toctext">Verbi</span>
asua :  <span class="toctext">Substantiivi</span>
osoittaa :  <span class="toctext">Verbi</span>
lehti :  <span class="toctext">Substantiivi</span>
siellä :  <span class="toctext">Adverbi</span>
aiemmin

tekijä :  <span class="toctext">Substantiivi</span>
tavallinen :  <span class="toctext">Adjektiivi</span>
kansa :  <span class="toctext">Substantiivi</span>
seitsemän :  <span class="toctext">Numeraali</span>
merkittävä :  <span class="toctext">Adjektiivi</span>
yö :  <span class="toctext">Substantiivi</span>
sana :  <span class="toctext">Substantiivi</span>
huolimatta :  <span class="toctext">Adverbi</span>
asti :  <span class="toctext">Postpositio</span>
vaihtoehto :  <span class="toctext">Substantiivi</span>
puuttua :  <span class="toctext">Verbi</span>
auttaa :  <span class="toctext">Verbi</span>
merkitys :  <span class="toctext">Substantiivi</span>
valmistua :  <span class="toctext">Verbi</span>
isä :  <span class="toctext">Substantiivi</span>
runsas :  <span class="toctext">Adjektiivi</span>
saakka :  <span class="toctext">Adverbi</span>
koskaan :  <span class="toctext">Adverbi</span>
kerätä :  <span class="toctext">Verbi</span>
yhtä :  <span class="toctext">Adverbi</span>
kuolla

ministeri :  <span class="toctext">Substantiivi</span>
varsinkin :  <span class="toctext">Adverbi</span>
tietysti :  <span class="toctext">Adverbi</span>
kustannus :  <span class="toctext">Substantiivi</span>
sairaala :  <span class="toctext">Substantiivi</span>
johtua :  <span class="toctext">Verbi</span>
syyskuu :  <span class="toctext">Substantiivi</span>
kansallinen :  <span class="toctext">Adjektiivi</span>
merkki :  <span class="toctext">Substantiivi</span>
välillä :  <span class="toctext">Adverbi</span>
tuomita :  <span class="toctext">Verbi</span>
keskus :  <span class="toctext">Substantiivi</span>
lopettaa :  <span class="toctext">Verbi</span>
lokakuu :  <span class="toctext">Substantiivi</span>
oppia :  <span class="toctext">Substantiivi</span>
yli :  <span class="toctext">Partikkeli</span>
sekunti :  <span class="toctext">Substantiivi</span>
lopulta :  <span class="toctext">Adverbi</span>
onni :  <span class="toctext">Substantiivi</span>
mikäli :  <span class="toctext">Adver

tuskin :  <span class="toctext">Adverbi</span>
suhtautua :  <span class="toctext">Verbi</span>
syödä :  <span class="toctext">Verbi</span>
teollisuus :  <span class="toctext">Substantiivi</span>
kilo :  <span class="toctext">Substantiivi</span>
käsittely :  <span class="toctext">Substantiivi</span>
peräti :  <span class="toctext">Adverbi</span>
ihmetellä :  <span class="toctext">Verbi</span>
vastustaa :  <span class="toctext">Verbi</span>
vaimo :  <span class="toctext">Substantiivi</span>
nauttia :  <span class="toctext">Verbi</span>
tunne :  <span class="toctext">Substantiivi</span>
säilyä :  <span class="toctext">Verbi</span>
helmikuu :  <span class="toctext">Substantiivi</span>
kohdata :  <span class="toctext">Verbi</span>
ote :  <span class="toctext">Substantiivi</span>
huhtikuu :  <span class="toctext">Substantiivi</span>
verrattuna :  <span class="toctext">Adverbi</span>
uhri :  <span class="toctext">Substantiivi</span>
idea :  <span class="toctext">Substantiivi</span>
suosia :  

In [177]:
with open('wordbank.json') as json_file:
    wordbank = json.load(json_file)

def practice():

    yle_url = 'https://yle.fi/uutiset/osasto/selkouutiset/'

    response=requests.get(yle_url)

    soup = BeautifulSoup(response.text, 'html.parser')

    results = soup.find_all("div", {"class": "text"})

    yle_articles = results[1].findAll(text=True)

    h3s = results[1].findAll('h3')
    headlines = []

    for h3 in h3s:
        temp = str(h3)
        temp = temp.replace('<h3>', '')
        temp = temp.replace('</h3>', '')
        headlines.append(temp)
        del temp

    unwanted_text = ['+ ', '–', '\n', '–\n', 'TV:n selkouutisilla on kesätauko. ', \
                      '(',')', ')\n', ';', '-"-\n', '; ']

    unwanted_text = unwanted_text + headlines

    article_list = [char for char in yle_articles if char not in unwanted_text]

    article_list = [sent for sent in article_list if not '\n' in sent]

    article_list = [sent_tokenize(sent) for sent in article_list]

    article_list = [sent for sublist in article_list for sent in sublist]

    article_list = [sent for sent in article_list if len(word_tokenize(sent))>1 ]

    pattern = RegexpTokenizer(r'\w+')

    # Split into sentences

    num = randint(0, len(article_list)-1)

    sentence = article_list[num]

    # Tokenize sentence, remove punctuation, numbers, and capitalized words (to avoid proper nouns).

    tokenized_sent = pattern.tokenize(sentence)

    tokenized_sent = [word for word in tokenized_sent if re.sub(r'[0-9]', '', word)]

    tokenized_sent = [word for word in tokenized_sent if not word[0].isupper()]

    num = randint(0, len(tokenized_sent)-1)

    word = tokenized_sent[num]

    fill_in_blank = sentence.replace(' '+word, ' _____ ')

    # if len(v.analyze(word)) > 0:
    if wordbank.get(word):

        word_baseform = wordbank.get(word)
        
        wikiurl = 'https://fi.wiktionary.org/wiki/' + word_baseform

        response=requests.get(wikiurl)

        soup = BeautifulSoup(response.text, 'html.parser')

        results = str(soup.find_all("span", {"class": "toctext"})[1])

        word_class = re.sub('<.*?>', '', results)

        # get the response in the form of html
        
        cases = []

        table_class="wikitable sortable jquery-tablesorter"

        response=requests.get(wikiurl)

        # parse data from the html into a beautifulsoup object

        soup = BeautifulSoup(response.text, 'html.parser')

        wikitable=soup.find('table',{'class':"wikitable"})

        if wikitable is None:
            table_contents = ''
        else: table_contents = wikitable.findAll(text=True)

        unwanted_chars = [word, '+ ', '–', '\n', '–\n', 'Taivutus\n', 'sijamuoto', 'nominatiivi', 'genetiivi', 'partitiivi', \
                          'akkusatiivi', 'sisäpaikallissijat', 'inessiivi', 'elatiivi', 'illatiivi', 'ulkopaikallissijat', \
                          'adessiivi', 'ablatiivi', 'allatiivi', 'essiivi', 'translatiivi', 'abessiivi', 'instruktiivi', \
                          'komitatiivi', 'positiivi', 'sijamuoto', 'yksikkö', 'monikko', 'kieliopilliset sijamuodot\n', \
                          'sisäpaikallissijat\n', 'ulkopaikallissijat\n', 'muut sijamuodot\n', 'omistusliite', 'monikko\n', \
                          'Positiivi', 'Komparatiivi', 'Superlatiivi', 'muut\n','(',')', ')\n', ';', 'Indikatiivi\n', 'preesens\n', \
                          'perfekti\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'minä\n', \
                          'sinä\n','hän\n','me\n','-"-\n','te', 'Te\n','he\n','passiivi','imperfekti\n','pluskvamperfekti\n', '; ']

        negatives = ['en ', 'olen ', 'en ole ', 'et ', 'olet ', 'et ole ', 'ei ',  'on ', 'ei ole ', 'emme ', 'olemme ', 'emme ole ',\
                     'ette ', 'olette ',  'ette ole ', 'ette ole ', 'eivät ', 'ovat ', 'eivät ole ', 'olin ', 'en ollut ', 'olit ', \
                     'et ollut ', 'oli ',  'ei ollut ', 'olimme ', 'emme olleet ', 'olitte ', 'ette olleet ', 'ette olleet ',  'olivat ', \
                     'eivät olleet ']

        if len(table_contents) > 0:
            cases = [char for char in table_contents if char not in unwanted_chars]

            cases = [char for char in cases if char not in negatives]

            cases = [''.join(e for e in case if e.isalnum()) for case in cases]

            cases = list(dict.fromkeys(cases))

        #set options from selected word and randomly selected cases
        if len(cases) > 0:
            num = randint(0, len(cases)-1)
            option_2 = cases[num].replace('-', '').replace('(', '').replace(')', '')

            num = randint(0, len(cases)-1)
            option_3 = cases[num].replace('-', '').replace('(', '').replace(')', '')

            options = [word, option_2, option_3]

            options = random.sample(options, len(options))

            return fill_in_blank, options, word
        else: return practice()
    else: return practice()

In [180]:
practice()


('Etelä-Suomessa sataa _____  vettä.',
 ['tiistaina', 'tiistait', 'tiistaitten'],
 'tiistaina')